# Rust's Nested Fixed Point Algorithm (NFXP)

본 포스트는 John Rust가 1987년 Econometrica에 게재한 [Optimal Replacement of GMC Bus Engines: An Empirical Model of Harold Zurcher](https://www.jstor.org/stable/1911259)에 기반하여 만들어졌습니다.

<br>

## 1. Introduction

Rust는 Harold Zurcher라는 한 버스 정비 담당자의 버스 엔진 교체 행태를 설명하기 위한 regenerative optimal stopping 모델을 제안했습니다. Rust의 귀무가설은 "Zurcher가 매 순간 버스 엔진을 교체할지 말지 결정하는 데 있어 optimal stopping rule을 따른다"인데, optimal stopping rule은 분석가들이 관찰하거나(예. 주행거리) 혹은 관찰할 수 없는 여러 state variable들의 함수일 것입니다.

만약 지금 당장 엔진을 교체한다면 즉각적으로 교체 비용이 발생할 것이고, 교체하지 않는다면 언제일지 모르는 미래에 버스가 고장나서 예상치 못한 비용을 지불해야 할 수 있습니다. 따라서 Zurcher는 각 시점마다 엔진을 교체할 때와 교체하지 않을 때 발생 가능한 비용 중 최소 비용을 부담하기 위해 선택을 내릴 것이고, 이에 대한 optimal stopping rule은 stochastic dynamic programming 문제의 solution으로 표현 가능합니다.

<br>

## 2. Model

$$\begin{equation}
    u(x,d,\theta) =
        \begin{cases}
            -RC - c(0,\theta) & \text{if } d=1 \\
            -c(x,\theta)      & \text{if } d=0
        \end{cases}
\end{equation}$$

$$\begin{equation}
    p(x_{t+1} | x_t,d_t,\theta) =
        \begin{cases}
            g(x_{t+1}-0   | \theta) & \text{if } d=1 \\
            g(x_{t+1}-x_t | \theta) & \text{if } d=0
        \end{cases}
\end{equation}$$

Optimal value function $V_{\theta}$는 아래 Bellman's equation의 unique solution입니다.

$$\begin{equation}
\begin{split}
    V_{\theta}(x,\epsilon) &= \max_{d \in D(x)} {[u(x,d,\theta) + \epsilon(d) 
                                               + \beta \int{V_{\theta}(x',\epsilon') \pi(dx',d\epsilon' | x,\epsilon,\theta)}]} \\
                           &= \max_{d \in D(x)} {[u(x,d,\theta) + \epsilon(d)
                                               + \beta EV_{\theta}(x,\epsilon,d)]}
\end{split}
\end{equation}$$

Conditional Independence Assumption (CIA)

$$\begin{equation}
    \pi(x_{t+1},\epsilon_{t+1} | x_t,\epsilon_t,d_t,\theta)
        = p(x_{t+1} | x_t,d_t,\theta) q(\epsilon_t | x_t,\theta)
\end{equation}$$

Conditional choice probability $P(d \vert x,\theta)$는 우리가 잘 아는 multinomial logit 공식을 통해 표현됩니다.

$$\begin{equation}
    P(d | x,\theta) = \frac{ \exp{\{ u(x,d,\theta) + \beta EV_{\theta}(x,d) \}} }
                           { \sum_{d' \in D(x)}{ \exp{\{ u(x,d',\theta) + \beta EV_{\theta}(x,d') \}} } }
\end{equation}$$

Contraction mapping

$$\begin{equation}
\begin{split}
    EV_{\theta}(x,d) &= T_{\theta}(EV_{\theta}(x,d)) \\
                     &= \int{\log{[ \sum_{d' \in D(x')}{\exp{\{ u(x',d',\theta) + \beta EV_{\theta}(x',d') \}}} ] p(dx'|x,d,\theta)}}
\end{split}
\end{equation}$$

<br>

## 3. Data

Ref: https://github.com/OpenSourceEconomics/rust-data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# raw data 읽어오기
g1 = np.loadtxt("./assets/05-NFXP/data/g870.asc").reshape(15,36)
g2 = np.loadtxt("./assets/05-NFXP/data/rt50.asc").reshape(4,60)
g3 = np.loadtxt("./assets/05-NFXP/data/t8h203.asc").reshape(48,81)
g4 = np.loadtxt("./assets/05-NFXP/data/a530875.asc").reshape(37,128)
g5 = np.loadtxt("./assets/05-NFXP/data/a530874.asc").reshape(12,137)
g6 = np.loadtxt("./assets/05-NFXP/data/a452374.asc").reshape(10,137)
g7 = np.loadtxt("./assets/05-NFXP/data/a530872.asc").reshape(18,137)
g8 = np.loadtxt("./assets/05-NFXP/data/a452372.asc").reshape(18,137)

In [3]:
pd.DataFrame(g2)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,2386.0,5.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,133099.0,136493.0,139512.0,140871.0,142747.0,145533.0,148197.0,149386.0,151441.0,153767.0
1,2387.0,5.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,128650.0,133058.0,136447.0,140778.0,143963.0,150465.0,152264.0,155275.0,158648.0,161748.0
2,2388.0,5.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,125177.0,127820.0,129933.0,132691.0,135919.0,138935.0,139956.0,142125.0,144834.0,147206.0
3,2389.0,5.0,81.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,114397.0,118541.0,123256.0,126303.0,130168.0,131540.0,134278.0,135206.0,138467.0,142009.0


In [4]:
data = None

g_id = 1
for g in [g1,g2,g3,g4,g5,g6,g7,g8]:
    # 헤더: 버스 ID, 구매 일자, 교체 시점 및 odometer readings, 최초 관측 일자 (일자들은 연/월 구분)
    head = pd.DataFrame(g[:,:11], columns=["id","purc_m","purc_y",
                                            "repl_1_m","repl_1_y","repl_1_odo",
                                            "repl_2_m","repl_2_y","repl_2_odo",
                                            "begin_m","begin_y"]).astype(int)
    
    # ym: year-month pairs
    y, m = int(head.begin_y.unique()), int(head.begin_m.unique())-1
    ym = []
    for i in range(pd.DataFrame(g[:,11:]).shape[1]):
        if m ==12:
            y += 1
            m = 1
        else:
            m += 1
        ym.append(str(y)+"-"+str(m))
    
    # 본문: 버스-일자-레벨 패널 데이터 생성
    g_ref = pd.DataFrame(g[:,11:], index=g[:,0].astype(int), columns=ym).stack().reset_index()
    g_ref.columns = ["id","date","odo"]
    g_ref.insert(loc=0, column="group", value=g_id) # 그룹 ID
    
    # 헤더 정보와 패널로 변환한 본문 결합
    head["date"] = head.apply(lambda row: str(row["repl_1_y"])+"-"+str(row["repl_1_m"]), axis=1)
    repl_1 = head[["id","date","repl_1_odo"]].query("repl_1_odo != 0")
    g_ref = pd.merge(g_ref, repl_1, how="left")

    head["date"] = head.apply(lambda row: str(row["repl_2_y"])+"-"+str(row["repl_2_m"]), axis=1)
    repl_2 = head[["id","date","repl_2_odo"]].query("repl_2_odo != 0")
    g_ref = pd.merge(g_ref, repl_2, how="left")

    # 종속변수 dtc: 엔진 교체 여부 (0 또는 1)
    g_ref["dtc"] = g_ref.apply(lambda row: 1 if row["repl_1_odo"]==row["repl_1_odo"] or row["repl_2_odo"]==row["repl_2_odo"] \
                               else 0, axis=1)
    g_ref["dtc_cum"] = g_ref.groupby("id").dtc.cumsum()
        
    # 설명변수 dtx: mileage at replacement
    tmp = g_ref.groupby("id").repl_1_odo.max().to_frame("odo_1").reset_index()
    g_ref = pd.merge(g_ref, tmp, how="outer")
    tmp = g_ref.groupby("id").repl_2_odo.max().to_frame("odo_2").reset_index()
    g_ref = pd.merge(g_ref, tmp, how="outer")
    
    def x_ref(row):
        if row["dtc"]==1 and row["dtc_cum"]==1:
            return row["repl_1_odo"]
        elif row["dtc"]==1 and row["dtc_cum"]==2:
            return row["repl_2_odo"] - row["odo_1"]
        elif row["dtc"]==0 and row["dtc_cum"]==0:
            return row["odo"]
        elif row["dtc"]==0 and row["dtc_cum"]==1:
            return row["odo"] - row["odo_1"]
        elif row["dtc"]==0 and row["dtc_cum"]==2:
            return row["odo"] - row["odo_2"]
    g_ref["dtx"] = g_ref.apply(x_ref, axis=1)
    
    # 불필요한 변수 제거
    g_ref.drop(columns=["odo","repl_1_odo","repl_2_odo","dtc_cum","odo_1","odo_2"], inplace=True)
    
    # concat
    if data is None:
        data = g_ref
    else:
        data = pd.concat([data, g_ref]).reset_index(drop=True)
    
    g_id += 1
    
data

,group,id,date,dtc,dtx
0,1,4403,83-5,0,504.0
1,1,4403,83-6,0,2705.0
2,1,4403,83-7,0,7345.0
3,1,4403,83-8,0,11591.0
4,1,4403,83-9,0,16057.0
...,...,...,...,...,...
15563,8,4256,85-1,0,138474.0
15564,8,4256,85-2,0,139320.0
15565,8,4256,85-3,0,140616.0
15566,8,4256,85-4,0,141292.0


In [5]:
# Table 2a
np.round(data.query("dtc==1").groupby("group").dtx.describe())

,count,mean,std,min,25%,50%,75%,max
group,,,,,,,,
3,27.0,199733.0,37459.0,124800.0,174550.0,204800.0,230650.0,273400.0
4,33.0,257336.0,65477.0,121300.0,215000.0,264100.0,292400.0,387300.0
5,11.0,245291.0,60258.0,118000.0,229600.0,250600.0,282250.0,322500.0
6,7.0,150786.0,61007.0,82400.0,107450.0,125500.0,197750.0,237200.0
7,27.0,208963.0,48981.0,121000.0,178650.0,207700.0,237200.0,331800.0
8,19.0,186700.0,43956.0,132000.0,162900.0,182100.0,188400.0,297500.0


In [6]:
# Table 2b
bus_nr = data.groupby("id").dtc.sum()
bus_nr = bus_nr[bus_nr == 0].index
np.round(data[data.id.isin(bus_nr)].groupby(["group","id"]).last().dtx.groupby("group").describe())

,count,mean,std,min,25%,50%,75%,max
group,,,,,,,,
1,15.0,100117.0,12929.0,65643.0,98580.0,101423.0,106280.0,120151.0
2,4.0,151182.0,8530.0,142009.0,145907.0,150486.0,155762.0,161748.0
3,21.0,250766.0,21325.0,199626.0,244599.0,254183.0,267988.0,280802.0
4,5.0,337222.0,17802.0,310910.0,326724.0,347549.0,348475.0,352450.0
5,1.0,326843.0,NaN,326843.0,326843.0,326843.0,326843.0,326843.0
6,3.0,265264.0,33332.0,232395.0,248376.0,264356.0,281698.0,299040.0
